## Vector Add in Numpy

In [ ]:
import numpy as np

np.random.seed(0)

n = 100
a = np.random.normal(size = n).astype(np.float32)
b = np.random.normal(size = n).astype(np.float32)


c = a+b


def vector_add(a,b,c):
    for i in range(n):
        c[i] = a[i]+b[i]

d = np.empty(shape= n, dtype = np.float32)
vector_add(a,b,d)

np.testing.assert_array_equal(c,d)

## Vector Add in TVM using TE

In [3]:
import tvm
import tvm.te as te

def vector_add(n):
    A = te.placeholder((n,), name = "a")
    B = te.placeholder((n,), name = "b")

    C = te.compute(A.shape, lambda i:A[i]+B[i], name = "C")

    return A,B,C

a,b,c = vector_add(100)

type(a), type(c)



(tvm.te.tensor.Tensor, tvm.te.tensor.Tensor)

## Create a Schedule to run the computation

In [4]:
c_schedule = te.create_schedule(c.op)

In [ ]:
# Lower the Schedule to TIR

tvm.lower(c_schedule,[a,b,c], name = "VectorAdd")


# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def VectorAdd(a: T.Buffer((100,), "float32"), b: T.Buffer((100,), "float32"), C: T.Buffer((100,), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for i in range(100):
            C[i] = a[i] + b[i]

In [ ]:
mod = tvm.build(c_schedule,[a,b,c], target= "llvm", name= "VectorAddBuild")


type(mod)

tvm.driver.build_module.OperatorModule